In [1]:
#Importing all the neccessary libraries
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
from nltk.stem.porter import PorterStemmer
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#from gensim.models import Word2Vec, KeyedVectors
import pickle
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from matplotlib.colors import ListedColormap
#import scikitplot.metrics as sciplot
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')        

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\devan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
filtered_data=pd.read_csv("Reviews.csv")


filtered_data.head()




,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
filtered_data.shape

(568454, 10)

In [4]:
#take a sample of 10k rows
filtered_data=filtered_data.sample(10000, random_state=42)

In [5]:
#Give reviews with Score = 3 a 'Neutral' tag and score > 3 a 'Positive' tag, and reviews with a score < 3 a 'Negative' tag.
filtered_data['SentimentPolarity'] = filtered_data['Score'].apply(lambda x : 'Positive' if x > 3 else ('Negative' if x < 3 else 'Neutral'))

In [6]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,SentimentPolarity
165256,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...,Positive
231465,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...,Positive
427827,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...,Neutral
433954,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin...",Negative
70260,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...,Positive


In [7]:
filtered_data['SentimentPolarity'].value_counts()

SentimentPolarity
Positive    7852
Negative    1398
Neutral      750
Name: count, dtype: int64

In [8]:
#Add class Labels that if sentiment is Positive then 2 else if Negative then 0 else 1
filtered_data['Class'] = filtered_data['SentimentPolarity'].apply(lambda x : 2 if x == 'Positive' else (0 if x == 'Negative' else 1))
filtered_data.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,SentimentPolarity,Class
165256,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...,Positive,2
231465,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...,Positive,2
427827,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...,Neutral,1
433954,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin...",Negative,0
70260,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...,Positive,2


In [9]:
filtered_data['Class'].value_counts()

Class
2    7852
0    1398
1     750
Name: count, dtype: int64

In [10]:
filtered_data.shape

(10000, 12)

In [11]:
#Removing the entries where HelpfulnessNumerator > HelpfulnessDenominator
filtered_data = filtered_data[filtered_data['HelpfulnessNumerator'] <= filtered_data['HelpfulnessDenominator']]
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,SentimentPolarity,Class
165256,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,Having tried a couple of other brands of glute...,Positive,2
231465,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,My cat loves these treats. If ever I can't fin...,Positive,2
427827,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,A little less than I expected. It tends to ha...,Neutral,1
433954,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,"First there was Frosted Mini-Wheats, in origin...",Negative,0
70260,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,and I want to congratulate the graphic artist ...,Positive,2


In [12]:
filtered_data.shape

(10000, 12)

In [13]:
#sentiment coloumn value count
filtered_data["SentimentPolarity"].value_counts()

SentimentPolarity
Positive    7852
Negative    1398
Neutral      750
Name: count, dtype: int64

In [14]:
#Droping unwanted coloumns
filtered_data = filtered_data.drop(['Id', 'ProductId',"Time","SentimentPolarity","UserId", 'Score', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Summary'], axis=1)
print("The shape of the sampled dataset after dropping unwanted columns : ", filtered_data.shape)
filtered_data.head()

The shape of the sampled dataset after dropping unwanted columns :  (10000, 2)


,Text,Class
165256,Having tried a couple of other brands of glute...,2
231465,My cat loves these treats. If ever I can't fin...,2
427827,A little less than I expected. It tends to ha...,1
433954,"First there was Frosted Mini-Wheats, in origin...",0
70260,and I want to congratulate the graphic artist ...,2


In [15]:
# function to remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))
# function to convert text to lowercase
def to_lowercase(text):
    return text.lower()
# function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
# function to lemmatize the text
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
# function to clean html tags
def clean_html(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)  
# function to preprocess the text
def preprocess_text(text):
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    text = clean_html(text)
    return text
# apply the preprocess_text function to the ReviewText column
filtered_data['cleaned_text'] = filtered_data['Text'].apply(preprocess_text)

In [16]:
filtered_data

,Text,Class,cleaned_text
165256,Having tried a couple of other brands of glute...,2,tried couple brand glutenfree sandwich cooky b...
231465,My cat loves these treats. If ever I can't fin...,2,cat love treat ever cant find house pop top bo...
427827,A little less than I expected. It tends to ha...,1,little less expected tends muddy taste expecte...
433954,"First there was Frosted Mini-Wheats, in origin...",0,first frosted miniwheats original size frosted...
70260,and I want to congratulate the graphic artist ...,2,want congratulate graphic artist putting entir...
...,...,...,...
477825,"The frame of this is what holds the weight, an...",2,frame hold weight hold nicely rest slightly fl...
414355,I ordered this kit just to try it since the pr...,2,ordered kit try since price good making wine y...
553964,This espresso is amazing. My wife would not dr...,2,espresso amazing wife would drink coffee produ...
471038,I was so happy when I decided to look for Runt...,2,happy decided look runt amazon able find store...


In [17]:
#drop the original text coloumn
filtered_data = filtered_data.drop(['Text'], axis=1)

In [18]:
filtered_data

,Class,cleaned_text
165256,2,tried couple brand glutenfree sandwich cooky b...
231465,2,cat love treat ever cant find house pop top bo...
427827,1,little less expected tends muddy taste expecte...
433954,0,first frosted miniwheats original size frosted...
70260,2,want congratulate graphic artist putting entir...
...,...,...
477825,2,frame hold weight hold nicely rest slightly fl...
414355,2,ordered kit try since price good making wine y...
553964,2,espresso amazing wife would drink coffee produ...
471038,2,happy decided look runt amazon able find store...


In [19]:
# tfidf vectorizer of cleaned_text coloumn
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_data['cleaned_text'])

tfidf_matrix.shape

(10000, 5000)

In [20]:
# train test split of tfidf_matrix and Class coloumn as target
X = tfidf_matrix
y = filtered_data['Class']  
X.shape, y.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 5000), (2000, 5000), (8000,), (2000,))

In [21]:
# take a sample of 2000 rows from X_train and y_train for faster training
X_train_sample = X_train[:2000]
y_train_sample = y_train[:2000]
X_train_sample.shape, y_train_sample.shape


((2000, 5000), (2000,))

In [22]:
# train all the models with the sample data and give full evaluation metrics on the test and train data



# Libraries List for Classification Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

models = {
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': MultinomialNB(),
    'random forest': RandomForestClassifier(),
    'logistic regression': LogisticRegression(),
    'decision tree': DecisionTreeClassifier()
}

print("Models list:", list(models.keys()))


model_performance = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred, average='weighted')
    recall = recall_score(y_test, y_test_pred, average='weighted')
    f1 = f1_score(y_test, y_test_pred, average='weighted')
    model_performance[model_name] = {
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }
model_performance



Models list: ['KNN', 'Naive Bayes', 'random forest', 'logistic regression', 'decision tree']


{'KNN': {'Train Accuracy': 0.824375,
  'Test Accuracy': 0.757,
  'Precision': 0.679579284768991,
  'Recall': 0.757,
  'F1 Score': 0.7051522347914997},
 'Naive Bayes': {'Train Accuracy': 0.796875,
  'Test Accuracy': 0.781,
  'Precision': 0.74209306647052,
  'Recall': 0.781,
  'F1 Score': 0.6907463425232748},
 'random forest': {'Train Accuracy': 0.999875,
  'Test Accuracy': 0.808,
  'Precision': 0.8137641971279373,
  'Recall': 0.808,
  'F1 Score': 0.7485003170926738},
 'logistic regression': {'Train Accuracy': 0.8735,
  'Test Accuracy': 0.8115,
  'Precision': 0.754384833869239,
  'Recall': 0.8115,
  'F1 Score': 0.7603905313916126},
 'decision tree': {'Train Accuracy': 1.0,
  'Test Accuracy': 0.732,
  'Precision': 0.7217990778937151,
  'Recall': 0.732,
  'F1 Score': 0.7267160898853097}}

In [23]:
# best model findings
best_model_name = max(model_performance, key=lambda x: model_performance[x]['F1 Score'])
best_model_performance = model_performance[best_model_name]
best_model_name, best_model_performance

('logistic regression',
 {'Train Accuracy': 0.8735,
  'Test Accuracy': 0.8115,
  'Precision': 0.754384833869239,
  'Recall': 0.8115,
  'F1 Score': 0.7603905313916126})

In [24]:
# hyperparameter tuning of the logistic regression using GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)
grid_search.best_params_
best_logistic_model = grid_search.best_estimator_
best_logistic_model


LogisticRegression(C=10, solver='saga')

In [25]:
# train model with logistic regression C=10, solver='saga' and give full evaluation metrics on the test and train data
final_model = LogisticRegression(C=10, solver='saga')
final_model.fit(X_train, y_train)
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

final_model_performance = {
    'Train Accuracy': train_accuracy,
    'Test Accuracy': test_accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

final_model_performance


{'Train Accuracy': 0.9785,
 'Test Accuracy': 0.817,
 'Precision': 0.7878361280648216,
 'Recall': 0.817,
 'F1 Score': 0.7955951473770243}

In [30]:
import pickle

# Train vectorizer
tfidf_vectorizer = TfidfVectorizer()  # do NOT change params later
X = tfidf_vectorizer.fit_transform(filtered_data['cleaned_text'])

# Train model
final_model.fit(X, y)   # assuming y is your labels

# Save model
with open("sentiment_model_tfidf.pkl", "wb") as f:
    pickle.dump(final_model, f)

# Save vectorizer (the one used above!)
with open("vectorizer_tfidf.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)
